# Testing 5-bit trigger codes w/ arduino & pyserial

In [1]:
import serial
import time

import numpy as np

### CONFIG

In [2]:
# list available serial ports
def list_serial_ports():
    import serial.tools.list_ports
    ports = serial.tools.list_ports.comports()
    print([port.device for port in ports])

In [3]:
# CHANGE THIS to match your Arduino port:
#   Windows: "COM3", "COM4", ...
#   macOS:   "/dev/tty.usbmodemXXXX" or "/dev/tty.usbserialXXXX"
#   Linux:   "/dev/ttyACM0", "/dev/ttyUSB0", etc.
list_serial_ports()


SERIAL_PORT = "COM6" #"/dev/ttyACM0"
BAUD_RATE = 115200


['COM4', 'COM6']


## Functions

In [4]:

def open_trigger_serial(port=SERIAL_PORT, baud_rate=BAUD_RATE):
    ser = serial.Serial(port, baud_rate, timeout=1)
    # Arduino auto-resets when serial opens; give it a moment
    time.sleep(2.0)
    
    print(f"Opened serial port {port} at {baud_rate} baud.")
    
    return ser


# Old - for constant pulse duration
# def send_trigger(ser, code):
#     """
#     Send a trigger code in [0, 31] to Arduino.
#     Arduino maps the 5 LSBs to pins (4, 6, 8, 10, 12)
#     and pulses them for PULSE_MS ms.
#     """
#     if not (0 <= code <= 31):
#         raise ValueError("Trigger code must be between 0 and 31 (inclusive).")

#     ser.write(bytes([code]))
#     ser.flush()   # push out immediately


def send_trigger(ser, code: int, duration_ms: int):
    """
    Send a trigger code (0–31) and pulse duration (ms, 0–65535) to Arduino.
    """
    if not (0 <= code <= 31):
        raise ValueError("code must be in 0–31")

    if not (0 <= duration_ms <= 65535):
        raise ValueError("duration_ms must be in 0–65535")

    low  = duration_ms & 0xFF
    high = (duration_ms >> 8) & 0xFF

    packet = bytes([code, low, high])
    
    # Clear any stale data first
    ser.reset_input_buffer()
    
    ser.write(packet)
    ser.flush()

## Explanation:
OpenBCI board has 5 input pins resulting in 32 (2^5) possible trigger codes (0-31). Each code is represented by the binary state of the 5 pins. For example:
- Code 0: 00000 (all pins LOW)
- Code 1: 00001 (pin 1 HIGH, others LOW)
- Code 31: 11111 (all pins HIGH)

The arduino is programmed to take an input code (0-32) via serial and set the corresponding pins HIGH/LOW based on the binary representation of that code.

## Tests

In [6]:
TRIG_DURATION_MS = 500


ser = open_trigger_serial()
time.sleep(3)  # small delay for Arduino reset

for _ in range(2):
    for code in range(1, 32):
        print(f"Sending trigger code: {code} -> {code:05b}") # (bytes: {bytes([code])})")
        send_trigger(ser, code, duration_ms=TRIG_DURATION_MS)
        time.sleep(TRIG_DURATION_MS/1000 + 0.5)  # wait for pulse + small gap
        
    time.sleep(2) 
        
ser.close()

Opened serial port COM6 at 115200 baud.
Sending trigger code: 1 -> 00001
Sending trigger code: 2 -> 00010
Sending trigger code: 3 -> 00011
Sending trigger code: 4 -> 00100
Sending trigger code: 5 -> 00101
Sending trigger code: 6 -> 00110
Sending trigger code: 7 -> 00111
Sending trigger code: 8 -> 01000
Sending trigger code: 9 -> 01001
Sending trigger code: 10 -> 01010
Sending trigger code: 11 -> 01011
Sending trigger code: 12 -> 01100
Sending trigger code: 13 -> 01101
Sending trigger code: 14 -> 01110
Sending trigger code: 15 -> 01111
Sending trigger code: 16 -> 10000
Sending trigger code: 17 -> 10001
Sending trigger code: 18 -> 10010
Sending trigger code: 19 -> 10011
Sending trigger code: 20 -> 10100
Sending trigger code: 21 -> 10101
Sending trigger code: 22 -> 10110
Sending trigger code: 23 -> 10111
Sending trigger code: 24 -> 11000
Sending trigger code: 25 -> 11001
Sending trigger code: 26 -> 11010
Sending trigger code: 27 -> 11011
Sending trigger code: 28 -> 11100
Sending trigger c

## Old - no duration

In [ ]:
# ser = open_trigger_serial()

# for _ in range(2):
#     for code in range(1, 32):
#         print(f"Sending trigger code: {code} -> {code:05b}") # (bytes: {bytes([code])})")
#         send_trigger(ser, code)
#         time.sleep(0.5)
        
#     time.sleep(5)
        
# ser.close()